# init and install

In [ ]:
!pip install --upgrade google-cloud-aiplatform

# question 1

In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

project_id = "peace-demo"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-pro-preview-0514")

prompt = """
This is that two people are discussing their favorite movies. and identify the names of the speakers. 
If the name cannot be identified, use Speaker A and Speaker B instead. Please exclude the names of movie characters and their friends.
Output first 10 sentences in the following format:
<timestamp> <speaker's name>: <content>.

After output the conversation, please summarize it with below format and the phone number, friend's name and time need to be in markdown bold format:
## Overall Summary
<summary and limit 100 words>

## Core Events <Three core events are described in detailed language>
1. <event>
2. <event>
3. <event>

## Their Friends' characteristics <Five Friends>
1. <name>: <Describe the characteristics>
...
"""

audio_file_uri = "gs://peace-demo-temp-us-central1/dialog_en_1hr.wav"
audio_file = Part.from_uri(audio_file_uri, mime_type="audio/mpeg")

# Generation Config
config = GenerationConfig(
    max_output_tokens=8192, temperature=0.6, top_p=0.5, top_k=24
)

contents = [audio_file, prompt]

response = model.generate_content(contents, generation_config=config)
print(response.text)

Okay, here is the transcript and summary of the conversation.

0:00:11 Speaker A: Okay, movies.
0:00:13 Speaker A: What's your favorite movie?
0:00:18 Speaker B: Uh probably Natural Born Killers.
0:00:22 Speaker A: Natural Born Killers is amazing.
0:00:25 Speaker A: I love I forgot his name.
0:00:29 Speaker A: Woody Harrelson.
0:00:30 Speaker B: Woody yeah, he's so cool.
0:00:33 Speaker B: Um
0:00:38 Speaker B: What movies do you not like?
0:00:43 Speaker A: Not like?
0:00:46 Speaker A: Uh okay, pretty much everything that Matt doesn't like.

## Overall Summary
Speaker A and Speaker B discuss their favorite movies, including Natural Born Killers, Talladega Nights, and Step Brothers. They also talk about horror films, with Speaker A expressing a preference for comedies. Speaker A mentions that they watched Lord of the Rings with **Matt**, who seems to have particular taste in movies. They plan to watch Return of the King that night. Speaker B recalls watching horror films with their dad

# question 2

In [21]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

project_id = "peace-demo"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-pro-preview-0514")

prompt = """
"Based on the dialogue in the screenshot, generate three different styles of replies using the same language as the conversation. Provide:

A formal response
A casual response
A humorous response

Maintain the tone and context of the original dialogue while adjusting the style for each reply."

Explanation:
Formal Response: This style uses polite and professional language, often suitable for official or business contexts.
Casual Response: This is more relaxed and conversational, appropriate for friends or informal settings.
Humorous Response: This style incorporates light-hearted or witty elements to make the reply entertaining.
"""

file_uri = "gs://peace-demo-temp-us-central1/chat_sceenshot/Picture1.png"
file = Part.from_uri(file_uri, mime_type="image/png")
    
contents = [file, prompt]


# Generation Config
config = GenerationConfig(
    max_output_tokens=8192, temperature=0.6, top_p=0.5, top_k=24,
    response_mime_type="application/text"
)

response = model.generate_content(contents)
print(response.text)

## Different Reply Styles to the Customer Complaint:

**Formal Response:**

"I understand your frustration, and I apologize for the inconvenience this delay is causing. I'll make sure to escalate this issue to the appropriate team immediately. They will have the necessary expertise to assist you further and ensure a swift resolution."


**Casual Response:** 

"No worries, I totally get how annoying that is!  It's definitely not cool for Amazon to be holding onto your money like that. Hang tight, I'm grabbing someone else from the team who can hopefully sort this out for you ASAP!"


**Humorous Response:** 

"Oh boy, battling Amazon's virtual assistant army to reach a human? I feel you! It's like trying to find a unicorn riding a dragon. But fear not,  I'll summon my inner superhero and get you a real person who can actually help. Hold on to your hats!" 



# question 3

In [5]:
! gsutil cp "gs://peace-demo-temp-us-central1/rag_synthesis_samples.txt" ./rag_synthesis_samples.txt

Copying gs://peace-demo-temp-us-central1/rag_synthesis_samples.txt...
- [1 files][ 17.8 KiB/ 17.8 KiB]                                                
Operation completed over 1 objects/17.8 KiB.                                     


In [16]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig

project_id = "peace-demo"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-pro-preview-0514")

prompt = """
Multiple Outputs for each example:
"Keywords":Keywords of the answer
"Answer":synthesized answer with keywords bold with markdown
"Referring":list referring the chunk number
"""

txt_file_uri = "./rag_synthesis_samples.txt"
with open(txt_file_uri, 'r') as file:
    chuncks = file.read()  # Read the entire file content
    
contents = [chuncks, prompt]

response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "Title": {
                "type": "string",
            },
            "Keywords": {
                "type": "string",
            },
            "Answer": {
                "type": "string",
            },
            "Referring": {
                "type": "string"
            }
        }
    }
}


# Generation Config
config = GenerationConfig(
    max_output_tokens=8192, temperature=0.6, top_p=0.5, top_k=24,
    response_mime_type="application/json", response_schema=response_schema
)

response = model.generate_content(contents)
print(response.text)

<EXAMPLE_1>
<Question>
my screen is too dark
</Question>
"Keywords": ['screen brightness', 'dark mode', 'bedtime mode', 'grayscale']
"Answer": To adjust your screen's brightness, you can access the **screen brightness** settings. If your device is in **dark mode**, try disabling it for a brighter display. Additionally, check if **bedtime mode** is activated, as it often reduces brightness. **Grayscale** mode can also make the screen appear darker.
"Referring": [0, 1, 4, 5, 9]
<EXAMPLE_2>
<Question>
find some file about Retrieval-Augmented Generation


</Question>
"Keywords": ['Retrieval-Augmented Generation', 'EvaluatingtheEffectivenessofRetrieval-Augmented.pdf', 'BenchmarkingLargeLanguageModelsinRetrieval-AugmentedGeneration.pdf']
"Answer": I found two files related to **Retrieval-Augmented Generation**:  "**EvaluatingtheEffectivenessofRetrieval-Augmented.pdf**" and "**BenchmarkingLargeLanguageModelsinRetrieval-AugmentedGeneration.pdf**".
"Referring": [0, 1]



# question 4,5,6

In [11]:
import vertexai
from vertexai.preview.vision_models import Image, ImageGenerationModel

project_id = "peace-demo"

vertexai.init(project=project_id, location="us-central1")

model = ImageGenerationModel.from_pretrained("imagegeneration@006")
#model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
base_img = Image.load_from_file(location="gs://peace-demo-temp-us-central1/cmp_samples/3.jpg")
mask_img = Image.load_from_file(location="gs://peace-demo-temp-us-central1/cmp_samples/3_mask.jpg")
output_file = "./cmp_samples_output_3.jpg"

#prompt="""
#Maintain the wall's color and texture exactly as they are. Ensure that the area where the clock was removed blends seamlessly with the rest of the wall, preserving its original appearance.
#"""

prompt="""
Ignore the people in the picture and don't make any changes
"""

images = model.edit_image(
    base_image=base_img,
    mask=mask_img,
    prompt=prompt,
    edit_mode="inpainting-remove",
    guidance_scale=21,
    seed=1,
    person_generation="allow_all",
    # Optional parameters
    negative_prompt="orange fence", # Describes the object being removed (i.e., "person"),
    number_of_images=1,
)

images[0].save(location=output_file, include_generation_parameters=False)

images[0].show()

FailedPrecondition: 400 Image editing failed with the following error: Project or the user is not allowlisted to turn off child detection filter

In [38]:
! gsutil cp "./cmp_samples_output_2.jpg" "gs://peace-demo-temp-us-central1/cmp_samples_output/2.jpg"

Copying file://./cmp_samples_output_1.jpg [Content-Type=image/jpeg]...
/ [1 files][  3.0 MiB/  3.0 MiB]                                                
Operation completed over 1 objects/3.0 MiB.                                      
